# Task 2 Comparing Performance
Check the amount of `time` it takes to run the code for calculating the `delta_e` of a single particle vs re-calculating the `delta_e` of the whole system for each step over **1000** steps

In [17]:
# Necessary Functions - Cell 1

import math

def calculate_LJ(r_ij):
    """
    The LJ interaction energy between two particles.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units.

    Parameters
    ----------
    r_ij : float
        The distance between the particles in reduced units.
    
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.
    """
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    pairwise_energy = 4 * (r12_term - r6_term)
    return pairwise_energy


def calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two 3D coordinates.

    Parameters
    ----------
    coord1, coord2: list
        The atomic coordinates

    Returns
    -------
    distance: float
        The distance between the two points.
    """

    distance = 0
    for i in range(3):
        dim_dist = coord1[i] - coord2[i]

        if box_length:
            dim_dist = dim_dist - box_length * round(dim_dist / box_length)

        dim_dist = dim_dist**2
        distance += dim_dist

    distance = math.sqrt(distance)
    return distance
    
def calculate_total_energy(coordinates, box_length, cutoff):
    """
    Calculate the total Lennard Jones energy of a system of particles.

    Parameters
    ----------
    coordinates : list
        Nested list containing particle coordinates.

    Returns
    -------
    total_energy : float
        The total pairwise Lennard Jones energy of the system of particles.
    """

    total_energy = 0

    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):

            dist_ij = calculate_distance(
                coordinates[i], coordinates[j], box_length=box_length
            )

            if dist_ij < cutoff:
                interaction_energy = calculate_LJ(dist_ij)
                total_energy += interaction_energy

    return total_energy

def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates

    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length

# Replace this with your group's function
def calculate_tail_correction(box_length, n_particles, cutoff):
    '''
    Calculates the tail correction to the internal energy found through the Lennard Jones Equation.
    
    Parameters
    ----------
    box_length : float
        length of the sides of the box created for the simulation
    n_particles : int
        number of particles created for the simulation
    cutoff : float
        cut-off distance
       
    Returns
    -------
    U_lrc : float
        this is the number corresponding with the energy of the particles outside of the defined cut-off limit    
    '''

    volume_of_box = math.pow((box_length),3)
    rc3_term = math.pow((1/cutoff),3)
    rc9_term = math.pow((rc3_term),3)
    inside_term = (((1/3)*rc9_term)-rc3_term)
    U_lrc = (8*math.pi*math.pow((n_particles),2)/(3*volume_of_box))*inside_term
    return U_lrc

In [18]:
# Necessary Functions - Cell 2

import random
def accept_or_reject(delta_e, beta):
    """Accept or reject based on an energy and beta (inverse temperature) measurement.
    
    Parameters
    ----------
    delta_e : float
        An energy change in reduced units
    beta: float
        Inverse temperature
        
    Returns
    -------
    bool
        True to accept move False to reject
    """
    if delta_e <= 0.0:
        accept = True
    else:
        random__number = random.random()
        p_acc = math.exp(-delta_e*beta)

        if random__number < p_acc:
            accept = True
        else:
            accept = False
    return accept

In [19]:
# Necessary Functions Cell 3
def calculate_pair_energy(coordinates, i_particle, box_length, cutoff):
    """
    Calculate the interaction energy of a particle with its environment (all other particles in the system).

    Parameters
    ----------
    coordinates: list
        The coordinates for all particles in the system
    i_particle: int
        The particle index for which to calculate the energy
    cutoff: float
        The simulation cutoff. Beyond this distance, the interactions are not calculated

    Returns
    -------
    e_total: float
        The pairwise interaction energy of the i_th particle with all other particles
    """
    e_total = 0.0
    i_position = coordinates[i_particle]

    num_atoms = len(coordinates)

    for j_particle in range(num_atoms):
        if i_particle != j_particle:

            J_position = coordinates[j_particle]
            r_ij = calculate_distance(i_position, J_position, box_length)

            if r_ij < cutoff:
                e_pair = calculate_LJ(r_ij)
                e_total += e_pair

    return e_total

In [28]:
# Prepare simulation
random.seed(0)

# Simulation Parameters
reduced_temperature = 0.9
num_steps = 1000
max_displacement = 0.1
cutoff = 3.0

freq = 100

# Calculated quantities
beta = 1 / reduced_temperature

# Get initial coordinates (initial system configuration)
coordinates, box_length = read_xyz("../data/sample_config1.txt")
num_particles = len(coordinates)

delta_energy = 0

total_energy = calculate_total_energy(coordinates, box_length, cutoff)
total_energy += calculate_tail_correction(box_length, num_particles, cutoff)
print(total_energy)

-4550.029078288015


In [29]:
# Original code that calculates delta_e for a single particle for comparison and times it
import time

start = time.time()
steps = []
energies = []
for step in range(num_steps):

    # 1. Randomly pick one of N particles
    random_particle = random.randrange(num_particles)

    # 2. Calculate the interaction energy of the selected particle with the system and store this value.
    current_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)

    # 3. Generate a random x, y, z displacement
    x_rand = random.uniform(-max_displacement, max_displacement)
    y_rand = random.uniform(-max_displacement, max_displacement)
    z_rand = random.uniform(-max_displacement, max_displacement)

    # 4. Modify the coordinate of Nth particle by the generated displacements.
    coordinates[random_particle][0] += x_rand
    coordinates[random_particle][1] += y_rand
    coordinates[random_particle][2] += z_rand

    # 5. Calculate the interaction energy of the selected particle with the system and store this value (using the new position).
    proposed_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)

    # 6. Calculate the change in energy and decide to accept or reject the change.
    delta_e = proposed_energy - current_energy

    accept = accept_or_reject(delta_e, beta)

    # 7. If accept, keep particle movement. If reject, move particle back.
    if accept:
        total_energy += delta_e
    else:
        # Move is not accepted, roll back coordinates
        coordinates[random_particle][0] -= x_rand
        coordinates[random_particle][1] -= y_rand
        coordinates[random_particle][2] -= z_rand

    # 8. Print the energy if the step is a multiple of freq
    if step % freq == 0:
        print (step, total_energy/num_particles, proposed_energy, delta_e, total_energy, random_particle)
        steps.append(step)
        energies.append(total_energy/num_particles)
end = time.time()

elapsed_time1 = end - start
print(elapsed_time1)


0 -5.6886200705459125 -9.761770922690493 -0.86697814871515 -4550.89605643673 394
100 -5.693007748937856 -12.936385912263814 -1.036816002941963 -4554.406199150285 113
200 -5.696362663512613 -11.039049724744887 0.4124310882060591 -4557.09013081009 231
300 -5.687784910313318 -11.230820432103396 -0.11214268737250954 -4550.227928250654 55
400 -5.69512358856122 -8.521754012610693 1.1706560592699464 -4556.098870848976 22
500 -5.696547953821493 -9.390784395202113 -0.21668753820635267 -4557.238363057194 394
600 -5.696319911336714 -9.388069798923262 2.1245007691946807 -4557.055929069371 117
700 -5.703142813763875 -11.681042287262857 0.5943086546836742 -4562.5142510111 439
800 -5.702783962071612 -10.748320799982533 0.3429987763861 -4562.22716965729 389
900 -5.6920418912910575 -12.615516379355451 -0.10976474604649944 -4553.633513032846 169
2.1731207370758057


In [30]:
random.seed(0)
import time

start = time.time()
steps = []
energies = []
for step in range(num_steps):

    # 1. Randomly pick one of N particles
    random_particle = random.randrange(num_particles)

    # Remove code that calculates paiswise energy for the random_particle
    # # 2. Calculate the interaction energy of the selected particle with the system and store this value.
    # current_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)

    # 3. Generate a random x, y, z displacement
    x_rand = random.uniform(-max_displacement, max_displacement)
    y_rand = random.uniform(-max_displacement, max_displacement)
    z_rand = random.uniform(-max_displacement, max_displacement)

    # 4. Modify the coordinate of Nth particle by the generated displacements.
    coordinates[random_particle][0] += x_rand
    coordinates[random_particle][1] += y_rand
    coordinates[random_particle][2] += z_rand

    # 5. Calculate the total energy of the new system, store it and add the tail correction
    proposed_energy = calculate_total_energy(coordinates, box_length, cutoff)
    proposed_energy += calculate_tail_correction(box_length, num_particles, cutoff)

    # 6. Calculate the change in energy and decide to accept or reject the change.
    delta_e = proposed_energy - total_energy

    accept = accept_or_reject(delta_e, beta)

    # 7. If accept, keep particle movement. If reject, move particle back.
    if accept:
        total_energy += delta_e
    else:
        # Move is not accepted, roll back coordinates
        coordinates[random_particle][0] -= x_rand
        coordinates[random_particle][1] -= y_rand
        coordinates[random_particle][2] -= z_rand

    # 8. Print the energy if the step is a multiple of freq
    if step % freq == 0:
        print (step, total_energy/num_particles, proposed_energy, delta_e, total_energy, random_particle)
        steps.append(step)
        energies.append(total_energy/num_particles)
end = time.time()

elapsed_time2 = end - start
print(elapsed_time2)


0 -5.678988215220732 -4543.190572176586 0.24496495246603445 -4543.190572176586 394
100 -5.65561963582331 -4523.27875434211 1.2169543165382493 -4524.495708658648 763
200 -5.648467043277033 -4513.553828766359 5.2198058552667135 -4518.773634621626 419
300 -5.632698052455066 -4506.158441964052 -0.917037312332468 -4506.158441964052 173
400 -5.635664779117788 -4506.631983416087 1.899839878144121 -4508.531823294231 205
500 -5.626805547722963 -4499.8974111762445 1.5470270021260148 -4501.444438178371 721
600 -5.622030054553756 -4496.254066921314 1.3699767216903638 -4497.624043643004 484
700 -5.6284998951287255 -4502.79991610298 -0.17479960607488465 -4502.79991610298 144
800 -5.632317499057508 -4505.853999246006 -0.36691439759852074 -4505.853999246006 107
900 -5.632622420658691 -4505.863681543009 0.234254983943174 -4506.097936526952 628
357.58480978012085


In [41]:
# Calculate percent time saved
(357.585-2.173)/357.585*100


99.39231231735111

## Reflection

### Approach
The goal is to determine what kind of improvement we made by choosing to only calculate the difference in pairwise interaction energy of a single particle with its environment for **1000** steps instead of recalculating the total energy of the system at each step. This was accomplished by using the `time` function to first calculate the elapsed time of the original function where the `delta_e` is calculated by moving a single particle and only calculating its pairwise interaction energies with the other particles. The code was then modified so that we no longer calculate the energy of that single particle; instead, the total energy of the system is calculated at each step. We then again use `time` to calculate the amount of time it takes to run that code.

### Observations
It took **2.172 seconds** to run the code for a single particle pairwise interaction calculation at each step. It took **357.584 seconds** for the code that recalculates the total energy of the system at each step. This means we shaved off 99.4% of the time by making the decision to only calculate a single particle's pairwise interaction energy instead of the whole system.